# 🍿 Aanbevelingen gebaseerd op gewogen beoordelingen

In dit notebook berekenen we gewogen beoordelingen voor elke film om robuustere aanbevelingen te maken. In plaats van alleen op de gemiddelde beoordeling te vertrouwen, die beïnvloed kan worden door een klein aantal beoordelingen, gebruiken we een gewogen benadering om ervoor te zorgen dat films met meer beoordelingen betrouwbaarder worden meegewogen.

De formule voor de gewogen beoordeling die we zullen gebruiken is geïnspireerd door de IMDB gewogen beoordelingsformule:

$$
W = \frac{Rv + Cm}{v + m}
$$

waarbij:

- $( W )$ = Gewogen Beoordeling
- $( R )$ = Gemiddelde beoordeling van de film
- $( v )$ = Aantal stemmen (of beoordelingen) voor de film
- $( m )$ = Minimum aantal stemmen vereist voor opname
- $( C )$ = Gemiddelde stem (of beoordeling) over alle films


### 3. Compare recommendations based on average rating and weighted ratings
Review the interface to note any significant differences with this method.

In [1]:
import pandas as pd

df = pd.read_csv('data/ratings.csv')
df = df.groupby(['movieId'])['rating'].agg(['mean', 'count']).reset_index()

# C is the mean vote across the whole report
C = df['mean'].mean()

movie_rating_counts = df.groupby('movieId').size()

# m is the minimum votes required to be listed in the chart;
m = df['count'].quantile(0.90)

# filter out all qualified movies into a new DataFrame
df = df.loc[df['count'] >= m]

def weighted_rating(x, m=m, C=C):
  v = x['count']
  R = x['mean']
  return ((R*v)+ (C*m))/(v+m)

df['weight'] = df.apply(weighted_rating, axis=1)

df = df.sort_values('weight', ascending=False).head(10)
df = df[['movieId', 'weight']]

df

,movieId,weight
277,318,4.337460
659,858,4.162494
2224,2959,4.161576
224,260,4.137000
46,50,4.123749
257,296,4.121515
461,527,4.119782
1938,2571,4.110118
897,1196,4.107505
314,356,4.095747


### 2. Sla de aanbevelingen op
Kies de top 10 boeken op basis van hun gewogen beoordelingen en sla deze aanbevelingen op als `recommendations-ratings-weight.csv`. Werk daarna de app-directory bij door het bestaande bestand te vervangen. Zorg ervoor dat het bestand de kolommen `movieId;weight` bevat.

In [4]:
df.to_csv('app/recommendations/recommendations-ratings-weight.csv', index=False)